In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Define paths
train_dir = 'training_data'
test_dir = 'testing_Data'
image_height = 32  
image_width = 32  
channels = 3

# Load datasets with data augmentation for the training set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=32,
    class_mode='sparse'
)

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=32,
    class_mode='sparse'
)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, channels)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(36, activation='softmax')
])

# Callbacks
checkpoint_callback = ModelCheckpoint(
    'cnn_improved.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    mode='max',
    verbose=1
)

reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr_callback]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')

# Load and evaluate the model with additional metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
import numpy as np

model = tf.keras.models.load_model('cnn_improved.keras')

# Get the true labels and predictions
y_true = np.concatenate([test_dataset[i][1] for i in range(len(test_dataset))])
y_pred = np.argmax(model.predict(test_dataset), axis=-1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)


Found 20628 images belonging to 36 classes.
Found 1008 images belonging to 36 classes.


c:\Users\Ashad InnO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15


c:\Users\Ashad InnO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


644/645 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.1744 - loss: 2.9314
Epoch 1: val_accuracy improved from -inf to 0.79464, saving model to cnn_improved.keras
645/645 ━━━━━━━━━━━━━━━━━━━━ 91s 137ms/step - accuracy: 0.1748 - loss: 2.9296 - val_accuracy: 0.7946 - val_loss: 0.7228 - learning_rate: 0.0010
Epoch 2/15
645/645 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5556 - loss: 1.4146
Epoch 2: val_accuracy improved from 0.79464 to 0.88194, saving model to cnn_improved.keras
645/645 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.5556 - loss: 1.4144 - val_accuracy: 0.8819 - val_loss: 0.4206 - learning_rate: 0.0010
Epoch 3/15
644/645 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6680 - loss: 1.0380
Epoch 3: val_accuracy did not improve from 0.88194
645/645 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.6680 - loss: 1.0378 - val_accuracy: 0.8700 - val_loss: 0.4012 - learning_rate: 0.0010
Epoch 4/15
643/645 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7191 - loss: 0.8873
Ep